In [ ]:
# 找到nan值来源 1
# KG为nan
# 给四元数作插值 替代nan值 （完成ing）
# 同时删减前后无法插值的 target （完成）
# 将初始化值设为每段target真值 （yes）
# 问题：跑得太慢
# 改进误差方程 改成批次计算 在批次中直接剔除 train_target为nan值的数（已完成）
# 运行NNTest成功

In [ ]:
# 目前状态：训练和验证分别来自两个数据集 
# 1.批次改进 1000个一批次进行处理 不能默认56批次（已完成）
# 2.将验证与训练在一个序列上实现 依据如下（不需要）
# 3.磁力计预测方程有误 R不用求转置 （已修改）
# 4.数据对齐 扒代码AI-IMU查看处理方法！（已完成）
# 将所有数据放置在列表中，1000个为一批次，初始化后去掉少于1000的；赋值给初始值，然后计算（已完成）
# 5.训练误差缩小到90度不再减少了，查找是否在数据处理中 有xyz轴取反的问题 （加上了四元数符号对齐）
#有可能问题        
# 6.将差异计算换乘欧拉角的形式 网络也相应改成n=3
# 7.能跑 一百多度 跑49次就会返回非法值
# 8.把状态都改成[batch_size, 3]的格式
# 9.loss改成两四元数转换

四元数乘法中的左乘和右乘决定了旋转的参照系：
左乘（pre-multiply）：意味着你从全局坐标系（惯性系）出发旋转。
右乘（post-multiply）：意味着旋转是在局部坐标系（物体自身的坐标系）进行的。

In [ ]:
# 将每个序列分成验证集和训练集（时间序列内切分）
# 这种方法将每个序列的数据根据时间顺序分成两部分：训练集（早期数据）和验证集（后期数据）。

# 特点：
# 时间连续性：保留每个序列的时间相关性。训练集用来训练模型，而验证集用来检查模型的预测能力，验证集永远包含时间更晚的部分。
# 适用场景：常用于时间序列预测任务，例如金融、气象、传感器数据等。这里重要的是时间依赖性，模型需要学会从历史数据中推断未来趋势。
# 对实验结果的影响：
# 时间依赖性保留：确保模型基于过去的数据预测未来，符合实际应用场景中模型的使用方式。
# 训练和测试的分布不同：训练数据和验证数据可能分布不同，因为验证集是序列后期的数据，这可能更具挑战性。序列后期的趋势可能与前期不同，因此更能测试模型的泛化能力。
# 模型的泛化性能更容易评估：这种划分方式与模型实际应用非常接近（预测未来），因此可以更真实地评估模型的性能。
# 2. 将不同的序列分成验证集和训练集（序列间切分）
# 这种方法将整个数据集的不同序列进行划分，一些序列用于训练集，另一些用于验证集。

# 特点：
# 序列独立性：训练集和验证集是完全不同的序列。模型只能从训练集中的序列学习特征，在验证集中的不同序列上评估模型性能。
# 适用场景：常用于任务中不同的序列代表不同的对象、实验条件、或环境（例如不同病人的医疗数据，不同区域的气象数据）。在这些情况下，序列之间的差异可能更重要，训练集和验证集的序列相互独立可以更好地测试模型在新序列上的泛化能力。
# 对实验结果的影响：
# 泛化性能的测试更严格：这种方式严格测试模型的泛化能力，尤其是在训练集和验证集的序列分布不同时。模型不能依赖某个特定序列的模式，而必须学习更加通用的特征。
# 可能更加现实：在一些应用中，未来使用模型时可能会面对完全新的序列（例如新病人或新地区的天气）。因此，这种划分方式更能反映模型在实际应用中遇到新数据时的表现。
# 对比和总结：
# 将每个序列分成训练集和验证集：

# 适合时间序列预测。
# 保留序列内部的时间依赖性，验证模型对时间趋势的预测能力。
# 更适合衡量模型在同一序列上的短期预测能力。
# 将不同序列分成训练集和验证集：

# 适合多个独立序列的任务。
# 强调模型的泛化能力，确保模型在面对新序列时仍能有效预测。
# 更适合测试模型在跨序列上的泛化能力，验证其是否能够处理新的数据模式。
# 选择哪种划分方式取决于实验目标：如果需要模型预测一个序列中的未来数据，第一种方法更合适；如果目标是模型能够应对新的序列，第二种方法会更加严格。

In [ ]:
    # pipeline
    # def plot_errors(self, roll_est, roll_gt, pitch_est, pitch_gt, yaw_est, yaw_gt):
    #     """
    #     绘制角度误差图。

    #     参数:
    #     roll_est, roll_gt: 横滚角估计值和真值
    #     pitch_est, pitch_gt: 俯仰角估计值和真值
    #     yaw_est, yaw_gt: 偏航角估计值和真值
    #     """
    #     plt.figure(figsize=(12, 10))

    #     # 横滚角
    #     plt.subplot(3, 1, 1)
    #     plt.plot(roll_est.detach().cpu().numpy(), label='Estimated Roll', linestyle='--')
    #     plt.plot(roll_gt.detach().cpu().numpy(), label='Ground Truth Roll', linestyle='-')
    #     plt.legend()
    #     plt.title('Roll Angle')

    #     # 俯仰角
    #     plt.subplot(3, 1, 2)
    #     plt.plot(pitch_est.detach().cpu().numpy(), label='Estimated Pitch', linestyle='--')
    #     plt.plot(pitch_gt.detach().cpu().numpy(), label='Ground Truth Pitch', linestyle='-')
    #     plt.legend()
    #     plt.title('Pitch Angle')

    #     # 偏航角
    #     plt.subplot(3, 1, 3)
    #     plt.plot(yaw_est.detach().cpu().numpy(), label='Estimated Yaw', linestyle='--')
    #     plt.plot(yaw_gt.detach().cpu().numpy(), label='Ground Truth Yaw', linestyle='-')
    #     plt.legend()
    #     plt.title('Yaw Angle')

    #     plt.tight_layout()
    #     plt.show()

    # def quaternion_multiply(self, q1, q2):
    #     """
    #     计算两个四元数的乘积。

    #     参数:
    #     q1, q2: 形状为 (N, T, 4) 的四元数张量

    #     返回:
    #     形状为 (N, T, 4) 的乘积四元数
    #     """
    #     w1, x1, y1, z1 = q1[:, :, 0], q1[:, :, 1], q1[:, :, 2], q1[:, :, 3]
    #     w2, x2, y2, z2 = q2[:, :, 0], q2[:, :, 1], q2[:, :, 2], q2[:, :, 3]

    #     w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    #     x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    #     y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    #     z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2

    #     return torch.stack([w, x, y, z], dim=2)

    # def normalize_quaternion(self, q):
    #     epsilon = 1e-8  # 小正数，避免出现 sqrt(0) 或负值
    #     q_square_sum = torch.sum(q**2, dim=2, keepdim=True)
        
    #     # 打印检查q_square_sum
    #     #print("q_square_sum:", q_square_sum)

    #     # 计算范数时添加 epsilon
    #     norm = torch.sqrt(q_square_sum + epsilon)
        
    #     return q / norm  # 归一化四元数

在这段代码中，当启用了 `CompositionLoss` 时，损失函数的计算方法结合了两种不同的损失。这是为了平衡两个不同的输出：`x_out_training_batch`（模型预测的状态）和 `y_hat`（经过非线性变换的模型输出），从而构建一个组合损失函数。

### 代码逻辑概述

1. **`self.args.CompositionLoss`** 为 `True`：
   - 使用组合损失函数，该损失函数由两个部分组成：一个是模型预测的状态与目标状态之间的误差，另一个是模型预测的观测值与真实观测值之间的误差。
   - **`alpha`** 是一个平衡系数，用于控制这两个损失项的权重。

2. **计算步骤**：
   - **`y_hat` 的计算**：  
     `y_hat` 是通过将模型输出的状态 `x_out_training_batch` 经过系统模型 `SysModel.h()` 的非线性映射得到的。这可以被视为将模型输出转换为观测空间。
   - **损失函数计算**：  
     当 `MaskOnState` 为 `True` 时，代码会使用掩码（`mask`）和 `train_lengthMask`，只计算有效的部分（即没有被填充的数据）的损失。`self.loss_fn` 是用于计算损失的损失函数，比如均方误差（MSE）。
     - 如果 `randomLength` 为 `True`，那么对于每个样本和掩码，分别计算损失并累加，最终取均值。
     - 否则，直接对批量数据进行整体损失的计算。

3. **损失公式**：
   当 `CompositionLoss` 启用时，损失的计算公式为：

   \[
   \text{MSE\_trainbatch\_linear\_LOSS} = \alpha \cdot \text{Loss}(x_{\text{out}}, \text{target}) + (1 - \alpha) \cdot \text{Loss}(y_{\hat}, y_{\text{training}})
   \]

   `Loss(x_out, target)`：表示预测的状态 `x_out_training_batch` 与真实目标状态 `train_target_batch` 之间的误差。
    `Loss(y_hat, y_training)`：表示模型预测的观测值 `y_hat` 与真实观测值 `y_training_batch` 之间的误差。
    通过 `alpha` 平衡这两部分损失，使得组合损失考虑了状态预测和观测值预测的准确性。

### 详细说明

#### (1) 组合损失的计算
```python
MSE_train_linear_LOSS[jj] = self.alpha * self.loss_fn(x_out_training_batch[jj,mask,train_lengthMask[index]], train_target_batch[jj,mask,train_lengthMask[index]]) + (1-self.alpha) * self.loss_fn(y_hat[jj,mask,train_lengthMask[index]], y_training_batch[jj,mask,train_lengthMask[index]])
```

在这种情况下，损失函数由两部分组成：
- 第一部分是状态预测 `x_out_training_batch` 和目标 `train_target_batch` 之间的误差。
- 第二部分是通过 `SysModel.h()` 得到的 `y_hat` 和真实观测值 `y_training_batch` 之间的误差。

组合损失使用了权重 `alpha` 来平衡这两部分损失。`alpha` 越接近 1，则状态预测的损失占更大比例，`alpha` 越接近 0，则观测值预测的损失占更大比例。

#### (2) `MaskOnState` 和 `randomLength` 的作用
- **`MaskOnState`**：表示是否在计算损失时仅对特定状态进行掩码计算，这通常用于处理填充的数据（例如，输入序列长度不同的情况），可以避免对填充部分计算损失。
- **`randomLength`**：表示输入数据的长度是否随机。如果是随机长度，那么损失计算会根据实际的有效长度进行。

#### (3) `y_hat` 的计算
`y_hat` 是通过系统模型的 `h()` 函数对 `x_out_training_batch` 进行非线性映射得到的：
```python
y_hat[:,:,t] = torch.squeeze(SysModel.h(torch.unsqueeze(x_out_training_batch[:,:,t])))
```
这一步的目的是将状态映射到观测空间，然后与实际的观测值进行对比，以计算组合损失中的第二项。

### 总结

在启用了 `CompositionLoss` 的情况下，损失函数不仅仅计算状态预测误差，还考虑了观测值预测的误差。通过 `alpha` 参数来平衡这两部分的损失，有助于模型在状态和观测的预测上同时保持准确性。

In [ ]:
      # def loss_fn(q1, q2):
        #     """
        #     计算两个四元数张量之间的角度差。
            
        #     参数:
        #     q1, q2: 形状为 (N, T, 4) 的四元数张量，N 为批次大小，T 为每批次的四元数数量。

        #     返回:
        #     损失值 (平均角度差)
        #     """

        #     # 四元数归一化，以确保它们是单位四元数
        #     q1 = normalize_quaternion(q1)
        #     q2 = normalize_quaternion(q2)

        #     # 四元数 q1 和 q2 的形状是 (N, T, 4)，每一行为 (w, x, y, z)
        #     # 计算四元数 q1 和 q2 之间的相对四元数
        #     q1_conj = torch.cat([q1[:, :, [0]], -q1[:, :, 1:]], dim=2)  # q1 的共轭
        #     q_rel = quaternion_multiply(q2, q1_conj)  # 计算 q2 * q1^(-1)

        #     # 计算相对四元数的旋转角度
        #     epsilon = 1e-7  # 防止数值不稳定

        #     # 非原地操作：先生成一个被 clamp 的新张量
        #     clamped_q_rel_0 = torch.clamp(q_rel[:, :, 0], -1.0 + epsilon, 1.0 - epsilon)  # 非原地操作

        #     # 使用被 clamp 的结果继续计算角度差
        #     angle_diff = 2 * torch.acos(clamped_q_rel_0)  # 角度差 (弧度)

        #     # 返回批次和每个四元数的平均角度差
        #     return angle_diff.mean()


        # def quaternion_multiply(q1, q2):
        #     """
        #     计算两个四元数的乘积。
            
        #     参数:
        #     q1, q2: 形状为 (N, T, 4) 的四元数张量
            
        #     返回:
        #     形状为 (N, T, 4) 的乘积四元数
        #     """
        #     w1, x1, y1, z1 = q1[:, :, 0], q1[:, :, 1], q1[:, :, 2], q1[:, :, 3]
        #     w2, x2, y2, z2 = q2[:, :, 0], q2[:, :, 1], q2[:, :, 2], q2[:, :, 3]

        #     w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
        #     x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
        #     y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
        #     z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2

        #     return torch.stack([w, x, y, z], dim=2)


        # def normalize_quaternion(q):
        #     """
        #     对四元数进行归一化。
            
        #     参数:
        #     q: 形状为 (N, T, 4) 的四元数张量
            
        #     返回:
        #     归一化后的四元数张量
        #     """
        #     norm = torch.sqrt(torch.sum(q**2, dim=2, keepdim=True))  # 在四元数维度 (最后一维) 进行求和
        #     return q / (norm + 1e-8)  # 避免除以零的情况

In [ ]:
#############################
###   load data DT case   ###
#############################
# 定义数据文件夹路径
data_folder = 'Simulations/data_mat/'

# 初始化存储 MSE 值的列表
mse_cv_linear_epoch_list = []
mse_train_linear_epoch_list = []
mse_cv_idx_opt_list = []
mse_cv_linear_opt_list = []
# 获取文件夹中所有 .mat 文件，并按字母/数字顺序排序
mat_files = sorted([f for f in os.listdir(data_folder) if f.endswith('.mat')])

# 循环遍历每个文件
# for mat_file in mat_files:
# 构建文件的完整路径
mat_file_path = os.path.join(data_folder, mat_file)

# 加载 .mat 文件
mat_data_input = scipy.io.loadmat(mat_file_path)

# 提取变量并转换为 torch 张量
imu_acc = torch.tensor(mat_data_input['imu_acc'])
imu_gyr = torch.tensor(mat_data_input['imu_gyr'])
imu_mag = torch.tensor(mat_data_input['imu_mag'])
imu_quat = torch.tensor(mat_data_input['opt_quat'], dtype=torch.double)

# 调用 clean_fix_data 函数
input_acc, input_gyr, input_mag, input_quat = clean_fix_data(
    imu_acc, imu_gyr, imu_mag, imu_quat
)

# 检查张量的大小（可选）
print(f'Processing file: {mat_file}')
print(f'acc: {input_acc.shape}, gyr: {input_gyr.shape}, mag: {input_mag.shape}, opt_quat: {input_quat.shape}')
  
# 这里可以选择保存结果或进行进一步处理


mat_data_cv= scipy.io.loadmat('Simulations/data_mat/37_disturbed_office_A.mat')
   #提取变量
cv_imu_acc = torch.tensor(mat_data_cv['imu_acc'])
cv_imu_gyr = torch.tensor(mat_data_cv['imu_gyr'])
cv_imu_mag = torch.tensor(mat_data_cv['imu_mag'])
cv_opt_quat = torch.tensor(mat_data_cv['opt_quat'])

cv_acc, cv_gyr, cv_mag, cv_quat = clean_fix_data(
  cv_imu_acc, cv_imu_gyr, cv_imu_mag, cv_opt_quat
)
# 检查张量的大小（可选）
print(f'acc: {cv_acc.shape}, gyr: {cv_gyr.shape}, mag: {cv_mag.shape},opt_quat: {cv_quat.shape}')

#  mat_test_cv= scipy.io.loadmat('Simulations/data_mat/03_undisturbed_slow_rotation_C.mat')
#  # 提取变量
#  test_imu_acc = torch.tensor(mat_test_cv['imu_acc'])
#  test_imu_gyr = torch.tensor(mat_test_cv['imu_gyr'])
#  test_imu_mag = torch.tensor(mat_test_cv['imu_mag'])
#  test_opt_quat = torch.tensor(mat_data_cv['opt_quat'])

#  test_acc, test_gyr, test_mag, test_quat = clean_fix_data(
#    test_imu_acc, test_imu_gyr, test_imu_mag, test_opt_quat
#  )
#  # 检查张量的大小（可选）
#  print(f'acc: {test_acc.shape}, gyr: {test_gyr.shape}, mag: {test_mag.shape},opt_quat: {test_quat.shape}')

sys_model = SystemModel(f, Q, h, R, args.T, args.T_test, m, n)# parameters for GT
sys_model.InitSequence(m1x_0, m2x_0)# x0 and P0

print("Data Load")

#训练集的输入数据
train_input_gyr = input_gyr
train_input_mag = input_mag
train_target = input_quat
#验证集的输入数据
cv_input_gyr = cv_gyr
cv_input_mag = cv_mag
cv_target = cv_quat
#测试集的输入数据
#  test_input_gyr = test_gyr
#  test_input_mag = test_acc
#  test_target = test_quat

N_T = train_input_gyr.size(0)
args.n_batch  = N_T//args.N_E #56

print("trainset size:",train_input_gyr.size())
print("cvset size:",cv_input_gyr.size())
#  print("testset size:",test_input_gyr.size())
print("batch size:",args.n_batch)
# Model with partial info
sys_model_partial = SystemModel(f, Q, h, R, args.T, args.T_test, m, n)
sys_model_partial.InitSequence(m1x_0, m2x_0)
# Model for 2nd pass
sys_model_pass2 = SystemModel(f, Q, h, R, args.T, args.T_test, m, n)# parameters for GT
sys_model_pass2.InitSequence(m1x_0, m2x_0)# x0 and P0